In [ ]:
from qm import QuantumMachinesManager, QopCaps
from qm.qua import *
from configuration import *

qmm = QuantumMachinesManager(host=qop_ip, port=None, cluster_name=cluster_name)
print(f"send program with config: {qmm.capabilities.supports(QopCaps.config_v2)}")

2025-09-16 12:05:17,760 - qm - INFO     - Starting session: e59e4fcd-4786-4f38-8f81-2e16de91e50b
2025-09-16 12:05:23,808 - qm - INFO     - Performing health check
2025-09-16 12:05:24,208 - qm - INFO     - Cluster healthcheck completed successfully.
send program with config: True


# Setting up the QMM and QUA program

In [2]:
with program() as hello_qua:
    set_dc_offset("q1_z", "single", 0.153)
    play("saturation", "q1_xy")

In [3]:
# Show backward compatiblity
qm = qmm.open_qm(full_config, close_other_machines=True)
job1 = qm.execute(hello_qua)
job1.wait_until("Done", timeout=60)

2025-09-16 12:05:24,270 - qm - WARNING  - No calibration_db set in octave config, skipping loading calibration data
2025-09-16 12:05:24,648 - qm - INFO     - Clearing queue
2025-09-16 12:05:25,436 - qm - INFO     - Adding program to queue.
2025-09-16 12:05:26,943 - qm - INFO     - Job f3992d2e-abc5-405f-a92d-8f7301f62c7d is done


# Temporary update of the DC offset from the controller_config

In [4]:
print("offset before execute:", qm.get_config()["controllers"][con]["fems"][lf_fem]["analog_outputs"][1]["offset"], "V")

offset before execute: 0.0 V


In [5]:
# Temporarily change offset by full config
controller_config["controllers"][con]["fems"][lf_fem]["analog_outputs"][1]["offset"] = 0.5
# Execute program with modified full_config
job2 = qm.execute(hello_qua, config=controller_config)
job2.wait_until("Done", timeout=60)

2025-09-16 12:05:30,945 - qm - INFO     - Clearing queue
2025-09-16 12:05:31,785 - qm - INFO     - Adding program to queue.
2025-09-16 12:05:33,246 - qm - INFO     - Job 36855419-c735-4abe-972e-ef42df3b0038 is done


In [ ]:
# The offset changed only happens during the execution of job2
print("offset after execute:", qm.get_config()["controllers"][con]["fems"][lf_fem]["analog_outputs"][1]["offset"], "V")

# Temporary update the pulse length from the logical config

In [7]:
# Change pulse length by logical_config
logical_config["pulses"]["saturation_pulse"]["length"] = 2 * saturation_len
# Execute program with modified logical_config
job3 = qm.execute(hello_qua, config=logical_config)
job3.wait_until("Done", timeout=60)

2025-09-16 12:05:44,518 - qm - INFO     - Clearing queue
2025-09-16 12:05:45,303 - qm - INFO     - Adding program to queue.
2025-09-16 12:05:46,791 - qm - INFO     - Job 98384eb4-2f1e-4a21-890b-5d4731b42ec2 is done


# Permanent update of the power

In [8]:
print(
    "full power before execute:",
    qm.get_config()["controllers"][con]["fems"][mw_fem]["analog_outputs"][1]["full_scale_power_dbm"],
    "dBm",
)

full power before execute: 1 dBm


In [9]:
# Change full scale power dBm for future job by controller_config
controller_config["controllers"][con]["fems"][mw_fem]["analog_outputs"][1]["full_scale_power_dbm"] = 4
qm.update_config(controller_config)
# Execute program with modified controller_config
job4 = qm.execute(hello_qua)
job4.wait_until("Done", timeout=60)

2025-09-16 12:05:48,068 - qm - INFO     - Clearing queue
2025-09-16 12:05:48,887 - qm - INFO     - Adding program to queue.
2025-09-16 12:05:50,423 - qm - INFO     - Job a1180960-0369-4175-80f3-027445da730b is done


In [10]:
# qm.update_config changed the config of the machine, so the change is permanent
print(
    "full power after execute:",
    qm.get_config()["controllers"][con]["fems"][mw_fem]["analog_outputs"][1]["full_scale_power_dbm"],
    "dBm",
)

full power after execute: 4 dBm


In [11]:
qm.close()

True